<a href="https://colab.research.google.com/github/JulianaDamurie/AprendizagemDeMaquina/blob/main/one_class_kc2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
# import
import time
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import KFold, StratifiedKFold
import math
import scipy
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager
from past.builtins import xrange
from collections import OrderedDict
from sklearn.utils import shuffle

In [71]:
class KNN:
    def __init__(self, k):
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def distance(self, X1, X2):
        distance = scipy.spatial.distance.euclidean(X1, X2)
    
    def fit_distance(self, X_test,k):
        final_output = []
        distances =  scipy.spatial.distance.cdist( self.X_train , X_test, 'euclidean')
        distances_win = []
        X_win = []

        for i in range(len(X_test)):
            d = []
            votes = []
            d_individuals = []
            x_individuals = []
            for j in range(len(self.X_train)):
                dist = np.absolute(distances[j][i])
                d.append([dist, j])
            d.sort()
            d = d[0:k]
            for d, j in d:
                votes.append(self.y_train[j])
                d_individuals.append(d)
                x_individuals.append(self.X_train[j])
            ans = Counter(votes).most_common(1)[0][0]
            final_output.append(ans)
            distances_win.append(d_individuals)
            X_win.append(x_individuals)
        return distances_win,X_win
    
    def predict(self,X_test,epsilon,k):
      final_output = []
      distances, x_win = self.fit_distance(X_test,k)

      for i in range(len(X_test)):
        d_intra, x_intra = self.fit_distance(x_win[i],2)

        calculate = distances[i][0]/(d_intra[0][1] + 0.0000000000000001)
        
        if (calculate<epsilon):
          final_output.append("false")
        else:
          final_output.append("true")
      return final_output
    def score(self, X_test, y_test):
        predictions = self.predict(X_test)
        return (predictions == y_test).sum() / len(y_test)


In [99]:
class KNN_modified:
    def __init__(self, k):
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def distance(self, X1, X2):
        distance = scipy.spatial.distance.euclidean(X1, X2)
    
    def fit_distance(self, X_test,k):
        final_output = []
        distances =  scipy.spatial.distance.cdist( self.X_train , X_test, 'euclidean')
        distances_win = []
        X_win = []

        for i in range(len(X_test)):
            d = []
            votes = []
            d_individuals = []
            x_individuals = []
            for j in range(len(self.X_train)):
                dist = np.absolute(distances[j][i])
                d.append([dist, j])
            d.sort()
            d = d[0:k]
            for d, j in d:
                votes.append(self.y_train[j])
                d_individuals.append(d)
                x_individuals.append(self.X_train[j])
            ans = Counter(votes).most_common(1)[0][0]
            final_output.append(ans)
            distances_win.append(d_individuals)
            X_win.append(x_individuals)
        return distances_win,X_win
    
    def predict(self,X_test,epsilon,k):
      final_output = []
      distances, x_win = self.fit_distance(X_test,k)
      for i in range(len(X_test)):
        d_intra, x_intra = self.fit_distance(x_win[i],2)

        sum_dist_intra = 0
        sum_dist_z = 0

        for j in range(len(d_intra)):
          sum_dist_intra += d_intra[j][1]
          sum_dist_z += distances[i][j]

        calculate = sum_dist_z/(sum_dist_intra + 0.0000000000000001)
        

        if (calculate<(epsilon)):
          final_output.append("false")
        else:
          final_output.append("true")
      return final_output
    def score(self, X_test, y_test):
        predictions = self.predict(X_test)
        return (predictions == y_test).sum() / len(y_test)


In [73]:
def sample_first_prows(data, perc):
    import pandas as pd
    return data.head(int(len(data)*(perc)))

def sample_last_prows(data, perc):
    import pandas as pd
    return data.tail(int(len(data)*(perc)))

In [74]:
# read first data set
dataset = False

if dataset:
  data, meta = arff.loadarff('/content/jm1.arff')

  df1 = pd.DataFrame(data)

  df1["defects"] = df1["defects"].astype(str).str.replace("b","")
  df1["defects"] = df1["defects"].astype(str).str.replace("'","")
  y_label = "defects"
else:
  data, meta = arff.loadarff('/content/kc2.arff')

  df1 = pd.DataFrame(data)

  df1["problems"] = df1["problems"].astype(str).str.replace("b","")
  df1["problems"] = df1["problems"].astype(str).str.replace("'","")
  df1["problems"] = df1["problems"].astype(str).str.replace("yes","true")
  df1["problems"] = df1["problems"].astype(str).str.replace("no","false")
  y_label = "problems"
df1 = df1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

df1

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,1.30,1.30,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,false
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,true
2,415.0,59.0,50.0,51.0,1159.0,8411.31,0.01,103.53,81.24,870848.58,2.80,48380.48,359.0,35.0,9.0,10.0,47.0,106.0,692.0,467.0,106.0,true
3,230.0,33.0,10.0,16.0,575.0,3732.82,0.03,39.82,93.74,148644.06,1.24,8258.00,174.0,15.0,34.0,5.0,23.0,67.0,343.0,232.0,65.0,true
4,175.0,26.0,12.0,13.0,500.0,3123.96,0.03,29.48,105.96,92103.07,1.04,5116.84,142.0,7.0,19.0,4.0,18.0,58.0,310.0,190.0,51.0,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,4.0,1.0,1.0,1.0,5.0,11.61,0.50,2.00,5.80,23.22,0.00,1.29,2.0,0.0,0.0,0.0,4.0,1.0,4.0,1.0,1.0,true
518,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,0.00,0.67,2.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,true
519,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,0.00,0.67,2.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,true
520,4.0,1.0,1.0,1.0,5.0,11.61,0.67,1.50,7.74,17.41,0.00,0.97,2.0,0.0,0.0,0.0,3.0,2.0,3.0,2.0,1.0,true


In [75]:
def separar_dados(df1,classe,percent):
  if(classe=='true'):
    n_classe = 'false'
  else:
    n_classe = 'true'
  scaler = StandardScaler()
  scaler.fit(df1.iloc[:, :-1])

  df_true = df1[df1[y_label].str.contains(classe, regex=False)]

  df_true.iloc[:, :-1] = scaler.transform(df_true.iloc[:, :-1])

  df_true = shuffle(df_true)

  df_false = df1[df1[y_label].str.contains(n_classe, regex=False)]

  df_false.iloc[:, :-1] = scaler.transform(df_false.iloc[:, :-1])

  df_false = shuffle(df_false)


  df_percent_false = sample_first_prows(df_false,percent)

  df_percent_false_test= sample_last_prows(df_false,1-percent)

  # divide into x_train, y_train, x_test and y_test

  df_test = pd.concat([df_percent_false_test,df_true])
  df_test = shuffle(df_test)

  x_df1 = np.array(df_false.iloc[:, :-1].values)
  y_df1 = np.array(df_false.iloc[:, 21].values)

  x_df2 = np.array(df_test.iloc[:, :-1].values)
  y_df2 = np.array(df_test.iloc[:, 21].values)

  return x_df1,y_df1,x_df2,y_df2

In [76]:
skf = StratifiedKFold(n_splits=5,shuffle=True)

In [100]:
list_percent = [0.3,0.4,0.5]
acc_peso = {}

for percent in list_percent:
  x_df1,y_df1,x_df2,y_df2 = separar_dados(df1,'true',percent)
  k = 1
  i = 0
  sum = 0 
  recall_true = 0
  recall_false = 0
  f1_true = 0
  precision_true = 0
  precision_false = 0
  f1_false = 0
  tn_sum =0
  fp_sum=0
  fn_sum=0
  tp_sum = 0
  tempo_init = time.time()

  for train_index, test_index in  skf.split(x_df2,y_df2):
    xdf1_test = x_df2[test_index]
    ydf1_test = y_df2[test_index]

    classifier = KNN(k)  
    classifier.fit(x_df1, y_df1) 

    tempo_treino = time.time() - tempo_init 
    tempo_init = time.time()

    ydf1_pred = classifier.predict(xdf1_test,1,k)

    tn, fp, fn, tp = confusion_matrix(ydf1_test, ydf1_pred).ravel() 


    tn_sum += tn
    fp_sum += fp
    fn_sum += fn
    tp_sum += tp

    sum += classification_report(ydf1_test, ydf1_pred,output_dict=True)['accuracy'] 
    recall_true += classification_report(ydf1_test, ydf1_pred,output_dict=True)['true']['recall']
    precision_true += classification_report(ydf1_test, ydf1_pred,output_dict=True)['true']['precision']
    f1_true += classification_report(ydf1_test, ydf1_pred,output_dict=True)['true']['f1-score']

    recall_false += classification_report(ydf1_test, ydf1_pred,output_dict=True)['false']['recall']
    precision_false += classification_report(ydf1_test, ydf1_pred,output_dict=True)['false']['precision']
    f1_false += classification_report(ydf1_test, ydf1_pred,output_dict=True)['false']['f1-score']


    print("k------",k,i)
    print(classification_report(ydf1_test, ydf1_pred))
    i+=1
  tempo = time.time() - tempo_init 
  acc_peso[k,0,percent] = [percent,sum/5,k,"OC-KNN",recall_true/5,precision_true/5,recall_false/5,precision_false/5,tempo_treino,tempo/5,tn_sum, fn_sum, tp_sum, fp_sum,f1_true/5,f1_false/5]




/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


k------ 1 0
              precision    recall  f1-score   support

       false       0.84      1.00      0.91        58
        true       1.00      0.50      0.67        22

    accuracy                           0.86        80
   macro avg       0.92      0.75      0.79        80
weighted avg       0.88      0.86      0.85        80

k------ 1 1
              precision    recall  f1-score   support

       false       0.85      1.00      0.92        58
        true       1.00      0.55      0.71        22

    accuracy                           0.88        80
   macro avg       0.93      0.77      0.81        80
weighted avg       0.89      0.88      0.86        80

k------ 1 2
              precision    recall  f1-score   support

       false       0.87      1.00      0.93        58
        true       1.00      0.57      0.73        21

    accuracy                           0.89        79
   macro avg       0.93      0.79      0.83        79
weighted avg       0.90      0.89     

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


k------ 1 0
              precision    recall  f1-score   support

       false       0.81      1.00      0.89        50
        true       1.00      0.45      0.62        22

    accuracy                           0.83        72
   macro avg       0.90      0.73      0.76        72
weighted avg       0.87      0.83      0.81        72

k------ 1 1
              precision    recall  f1-score   support

       false       0.82      1.00      0.90        50
        true       1.00      0.48      0.65        21

    accuracy                           0.85        71
   macro avg       0.91      0.74      0.77        71
weighted avg       0.87      0.85      0.83        71

k------ 1 2
              precision    recall  f1-score   support

       false       0.81      1.00      0.89        50
        true       1.00      0.43      0.60        21

    accuracy                           0.83        71
   macro avg       0.90      0.71      0.75        71
weighted avg       0.86      0.83     

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


k------ 1 0
              precision    recall  f1-score   support

       false       0.88      1.00      0.93        42
        true       1.00      0.71      0.83        21

    accuracy                           0.90        63
   macro avg       0.94      0.86      0.88        63
weighted avg       0.92      0.90      0.90        63

k------ 1 1
              precision    recall  f1-score   support

       false       0.78      1.00      0.88        42
        true       1.00      0.43      0.60        21

    accuracy                           0.81        63
   macro avg       0.89      0.71      0.74        63
weighted avg       0.85      0.81      0.78        63

k------ 1 2
              precision    recall  f1-score   support

       false       0.80      1.00      0.89        41
        true       1.00      0.55      0.71        22

    accuracy                           0.84        63
   macro avg       0.90      0.77      0.80        63
weighted avg       0.87      0.84     

In [101]:
k_list = [1,2,3,5,7]
list_percent = [0.3,0.4,0.5]


for percent in list_percent:
  x_df1,y_df1,x_df2,y_df2 = separar_dados(df1,'true',percent)
  for k in k_list :
    i = 0
    sum = 0 
    recall_true = 0
    recall_false = 0
    f1_true = 0
    precision_true = 0
    precision_false = 0
    f1_false = 0
    tn_sum =0
    fp_sum=0
    fn_sum=0
    tp_sum = 0
    tempo_init = time.time()

    for train_index, test_index in  skf.split(x_df2,y_df2):
      xdf1_test = x_df2[test_index]
      ydf1_test = y_df2[test_index]

      classifier = KNN_modified(k)  
      classifier.fit(x_df1, y_df1) 

      tempo_treino = time.time() - tempo_init 
      tempo_init = time.time()

      ydf1_pred = classifier.predict(xdf1_test,1,k)

      tn, fp, fn, tp = confusion_matrix(ydf1_test, ydf1_pred).ravel() 


      tn_sum += tn
      fp_sum += fp
      fn_sum += fn
      tp_sum += tp

      sum += classification_report(ydf1_test, ydf1_pred,output_dict=True)['accuracy'] 
      recall_true += classification_report(ydf1_test, ydf1_pred,output_dict=True)['true']['recall']
      precision_true += classification_report(ydf1_test, ydf1_pred,output_dict=True)['true']['precision']
      f1_true += classification_report(ydf1_test, ydf1_pred,output_dict=True)['true']['f1-score']

      recall_false += classification_report(ydf1_test, ydf1_pred,output_dict=True)['false']['recall']
      precision_false += classification_report(ydf1_test, ydf1_pred,output_dict=True)['false']['precision']
      f1_false += classification_report(ydf1_test, ydf1_pred,output_dict=True)['false']['f1-score']


      print("k------",k,i)
      print(classification_report(ydf1_test, ydf1_pred))
      i+=1
    tempo = time.time() - tempo_init 
    acc_peso[k,1,percent] = [percent,sum/5,k,"OC-KNNM",recall_true/5,precision_true/5,recall_false/5,precision_false/5,tempo_treino,tempo/5,tn_sum, fn_sum, tp_sum, fp_sum,f1_true/5,f1_false/5]




/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


k------ 1 0
              precision    recall  f1-score   support

       false       0.85      1.00      0.92        58
        true       1.00      0.55      0.71        22

    accuracy                           0.88        80
   macro avg       0.93      0.77      0.81        80
weighted avg       0.89      0.88      0.86        80

k------ 1 1
              precision    recall  f1-score   support

       false       0.89      1.00      0.94        58
        true       1.00      0.68      0.81        22

    accuracy                           0.91        80
   macro avg       0.95      0.84      0.88        80
weighted avg       0.92      0.91      0.91        80

k------ 1 2
              precision    recall  f1-score   support

       false       0.83      1.00      0.91        58
        true       1.00      0.43      0.60        21

    accuracy                           0.85        79
   macro avg       0.91      0.71      0.75        79
weighted avg       0.87      0.85     

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


k------ 1 0
              precision    recall  f1-score   support

       false       0.85      1.00      0.92        50
        true       1.00      0.59      0.74        22

    accuracy                           0.88        72
   macro avg       0.92      0.80      0.83        72
weighted avg       0.89      0.88      0.86        72

k------ 1 1
              precision    recall  f1-score   support

       false       0.82      1.00      0.90        50
        true       1.00      0.48      0.65        21

    accuracy                           0.85        71
   macro avg       0.91      0.74      0.77        71
weighted avg       0.87      0.85      0.83        71

k------ 1 2
              precision    recall  f1-score   support

       false       0.85      1.00      0.92        50
        true       1.00      0.57      0.73        21

    accuracy                           0.87        71
   macro avg       0.92      0.79      0.82        71
weighted avg       0.89      0.87     

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


k------ 1 1
              precision    recall  f1-score   support

       false       0.81      1.00      0.89        42
        true       1.00      0.52      0.69        21

    accuracy                           0.84        63
   macro avg       0.90      0.76      0.79        63
weighted avg       0.87      0.84      0.82        63

k------ 1 2
              precision    recall  f1-score   support

       false       0.80      1.00      0.89        41
        true       1.00      0.55      0.71        22

    accuracy                           0.84        63
   macro avg       0.90      0.77      0.80        63
weighted avg       0.87      0.84      0.83        63

k------ 1 3
              precision    recall  f1-score   support

       false       0.85      1.00      0.92        41
        true       1.00      0.68      0.81        22

    accuracy                           0.89        63
   macro avg       0.93      0.84      0.87        63
weighted avg       0.91      0.89     

In [102]:
df = pd.DataFrame.from_dict(acc_peso,orient='index',columns=['percent','taxa de acerto','valor de k','k-NN',"recall true","precision true","recall false","precision false",'tempo treino',"tempo teste","True Negative", "False Negative", "True Positive", "False Positive", "F1-Mesure True","F1-Mesure False"])

In [104]:
df

,percent,taxa de acerto,valor de k,k-NN,recall true,precision true,recall false,precision false,tempo treino,tempo teste,True Negative,False Negative,True Positive,False Positive,F1-Mesure True,F1-Mesure False
"(1, 0, 0.3)",0.3,0.879146,1,OC-KNN,0.551948,1.000000,1.000000,0.858107,0.210619,0.042613,290,48,59,0,0.710873,0.923604
"(1, 0, 0.4)",0.4,0.865258,1,OC-KNN,0.551082,1.000000,1.000000,0.840086,0.183357,0.036625,249,48,59,0,0.702861,0.912669
"(1, 0, 0.5)",0.5,0.846953,1,OC-KNN,0.550216,1.000000,1.000000,0.814025,0.163259,0.032098,207,48,59,0,0.700350,0.896829
"(1, 1, 0.3)",0.3,0.879019,1,OC-KNNM,0.550216,1.000000,1.000000,0.858638,0.203401,0.042185,290,48,59,0,0.705312,0.923769
"(2, 1, 0.3)",0.3,0.919335,2,OC-KNNM,0.700000,1.000000,1.000000,0.901019,0.293799,0.057622,290,32,75,0,0.821635,0.947829
"(3, 1, 0.3)",0.3,0.712975,3,OC-KNNM,0.795238,0.487283,0.682759,0.899344,0.375587,0.075737,198,22,85,92,0.602153,0.774749
"(5, 1, 0.3)",0.3,0.508734,5,OC-KNNM,0.925108,0.346240,0.355172,0.932155,0.517863,0.105058,103,8,99,187,0.503609,0.512593
"(7, 1, 0.3)",0.3,0.450759,7,OC-KNNM,0.945022,0.322995,0.268966,0.939394,0.687566,0.138951,78,6,101,212,0.480957,0.414783
"(1, 1, 0.4)",0.4,0.865141,1,OC-KNNM,0.550649,1.000000,1.000000,0.838550,0.196581,0.036905,249,48,59,0,0.709130,0.912148
"(2, 1, 0.4)",0.4,0.910290,2,OC-KNNM,0.701299,1.000000,1.000000,0.888566,0.261449,0.051245,249,32,75,0,0.817254,0.940412
